<h2> Maritime Risk: Piracy </h2>

<h3> Data Cleaning </h3>
Source: Kaggle - Global Maritime Pirate Attacks (1993–2020)

https://www.kaggle.com/datasets/n0n5ense/global-maritime-pirate-attacks-19932020 

In [65]:
import pandas as pd
import numpy as np

# Paths
PIRATE_PATH     = "piracy/raw_data_archive/pirate_attacks.csv"
CODES_PATH      = "piracy/raw_data_archive/country_codes.csv"          # for region lookup
INDICATORS_PATH = "piracy/raw_data_archive/country_indicators.csv"     # optional

OUT_PATH        = "piracy/cleaned_data_output/cleaned_pirate_attacks.csv"
MIN_ROWS        = 5000  # keep at least this many rows after cleaning

pd.set_option("display.max_columns", 100)


In [66]:
# Load CSVs
pir = pd.read_csv(PIRATE_PATH)
codes = pd.read_csv(CODES_PATH)

# Normalize headers once
pir.columns = [c.strip().lower().replace(" ", "_") for c in pir.columns]
codes.columns = [c.strip().lower().replace(" ", "_") for c in codes.columns]

# Indicators are optional
try:
    ind = pd.read_csv(INDICATORS_PATH)
    ind.columns = [c.strip().lower().replace(" ", "_") for c in ind.columns]
except Exception:
    ind = None

pir.head(), pir.shape, codes.head(), ind.head() if ind is not None else "No indicators loaded"


(         date time  longitude   latitude attack_type  \
 0  1993-01-02  NaN   116.9667  19.700000         NaN   
 1  1993-01-04  NaN   116.0000  22.350000         NaN   
 2  1993-01-06  NaN   115.2500  19.670000         NaN   
 3  1993-01-08  NaN   124.5833  29.900000         NaN   
 4  1993-01-12  NaN   120.2667  18.133333         NaN   
 
          location_description nearest_country eez_country  shore_distance  \
 0  Hong Kong - Luzon - Hainan             CHN         TWN      357.502373   
 1  Hong Kong - Luzon - Hainan             CHN         CHN       47.431573   
 2  Hong Kong - Luzon - Hainan             CHN         TWN      280.811871   
 3              East China Sea             CHN         CHN      209.923396   
 4  Hong Kong - Luzon - Hainan             PHL         PHL       22.027332   
 
    shore_longitude  shore_latitude attack_description           vessel_name  \
 0       115.825956       22.746644                NaN      Mv Cosmic Leader   
 1       115.825956       

In [67]:
pir.head(7)

,date,time,longitude,latitude,attack_type,location_description,nearest_country,eez_country,shore_distance,shore_longitude,shore_latitude,attack_description,vessel_name,vessel_type,vessel_status,data_source
0,1993-01-02,NaN,116.9667,19.700000,NaN,Hong Kong - Luzon - Hainan,CHN,TWN,357.502373,115.825956,22.746644,NaN,Mv Cosmic Leader,NaN,NaN,mappingpiracy
1,1993-01-04,NaN,116.0000,22.350000,NaN,Hong Kong - Luzon - Hainan,CHN,CHN,47.431573,115.825956,22.746644,NaN,Mv Tricolor Star III,NaN,NaN,mappingpiracy
2,1993-01-06,NaN,115.2500,19.670000,NaN,Hong Kong - Luzon - Hainan,CHN,TWN,280.811871,114.302501,22.044867,NaN,Mv Arktis Star,NaN,NaN,mappingpiracy
3,1993-01-08,NaN,124.5833,29.900000,NaN,East China Sea,CHN,CHN,209.923396,122.409679,29.911200,NaN,Ussurijsk,NaN,NaN,mappingpiracy
4,1993-01-12,NaN,120.2667,18.133333,NaN,Hong Kong - Luzon - Hainan,PHL,PHL,22.027332,120.470063,18.091010,NaN,Mv Chennai Nermai,NaN,NaN,mappingpiracy
5,1993-01-13,NaN,101.8500,9.717300,NaN,Gulf of Thailand. South China Sea,KHM,KHM,184.796641,103.292166,10.583157,NaN,Emelyan Yaroslavskiy,NaN,NaN,mappingpiracy
6,1993-01-25,NaN,106.8667,8.566667,NaN,South China Sea,VNM,VNM,27.437344,106.655772,8.698879,NaN,Mv Indira Ghandhi,NaN,NaN,mappingpiracy


In [68]:
print("Missing values:\n", pir.isna().sum().sort_values(ascending=False).head(12))
print("\nDtypes:\n", pir.dtypes)

Missing values:
 time                    6362
attack_description      6338
vessel_type             6338
vessel_name             1432
vessel_status            912
eez_country              295
attack_type              120
nearest_country           19
location_description       8
date                       0
longitude                  0
latitude                   0
dtype: int64

Dtypes:
 date                     object
time                     object
longitude               float64
latitude                float64
attack_type              object
location_description     object
nearest_country          object
eez_country              object
shore_distance          float64
shore_longitude         float64
shore_latitude          float64
attack_description       object
vessel_name              object
vessel_type              object
vessel_status            object
data_source              object
dtype: object


In [69]:
# Type Conversions
# Date
pir["date"] = pd.to_datetime(pir["date"], errors="coerce")

# Time (keep raw time as a clean string)
if "time" in pir.columns:
    pir["time"] = pir["time"].astype(str).str.strip().replace(
        {"": np.nan, "nan": np.nan, "NaN": np.nan}
    )

# Make a single timestamp using date + time (fallback to 00:00:00 if time is missing)
pir["timestamp"] = pd.to_datetime(
    pir["date"].dt.strftime("%Y-%m-%d") + " " + pir["time"].fillna("00:00:00"),
    errors="coerce",
    utc=True
)

# Numeric columns
for col in ["longitude", "latitude", "shore_distance", "shore_longitude", "shore_latitude"]:
    if col in pir.columns:
        pir[col] = pd.to_numeric(pir[col], errors="coerce")

pir.dtypes


date                         datetime64[ns]
time                                 object
longitude                           float64
latitude                            float64
attack_type                          object
location_description                 object
nearest_country                      object
eez_country                          object
shore_distance                      float64
shore_longitude                     float64
shore_latitude                      float64
attack_description                   object
vessel_name                          object
vessel_type                          object
vessel_status                        object
data_source                          object
timestamp               datetime64[ns, UTC]
dtype: object

In [70]:
# type conversions check
print(pir[["date","time","timestamp"]].head(10))
print("timestamp dtype:", pir["timestamp"].dtype)

        date time                 timestamp
0 1993-01-02  NaN 1993-01-02 00:00:00+00:00
1 1993-01-04  NaN 1993-01-04 00:00:00+00:00
2 1993-01-06  NaN 1993-01-06 00:00:00+00:00
3 1993-01-08  NaN 1993-01-08 00:00:00+00:00
4 1993-01-12  NaN 1993-01-12 00:00:00+00:00
5 1993-01-13  NaN 1993-01-13 00:00:00+00:00
6 1993-01-25  NaN 1993-01-25 00:00:00+00:00
7 1993-01-26  NaN 1993-01-26 00:00:00+00:00
8 1993-01-26  NaN 1993-01-26 00:00:00+00:00
9 1993-01-26  NaN 1993-01-26 00:00:00+00:00
timestamp dtype: datetime64[ns, UTC]


In [71]:
# Drop exact duplicates
pir = pir.drop_duplicates()

# Valid coordinates
pir = pir[(pir["longitude"].between(-180, 180)) & (pir["latitude"].between(-90, 90))]

# Keep dataset range 1993–2020 (as per description)
pir = pir[(pir["date"].dt.year >= 1993) & (pir["date"].dt.year <= 2020)]

# Require core fields
pir = pir.dropna(subset=["date", "longitude", "latitude"]).reset_index(drop=True)

print("Rows after basic filters:", len(pir))
pir.head(3)


Rows after basic filters: 7491


,date,time,longitude,latitude,attack_type,location_description,nearest_country,eez_country,shore_distance,shore_longitude,shore_latitude,attack_description,vessel_name,vessel_type,vessel_status,data_source,timestamp
0,1993-01-02,NaN,116.9667,19.70,NaN,Hong Kong - Luzon - Hainan,CHN,TWN,357.502373,115.825956,22.746644,NaN,Mv Cosmic Leader,NaN,NaN,mappingpiracy,1993-01-02 00:00:00+00:00
1,1993-01-04,NaN,116.0000,22.35,NaN,Hong Kong - Luzon - Hainan,CHN,CHN,47.431573,115.825956,22.746644,NaN,Mv Tricolor Star III,NaN,NaN,mappingpiracy,1993-01-04 00:00:00+00:00
2,1993-01-06,NaN,115.2500,19.67,NaN,Hong Kong - Luzon - Hainan,CHN,TWN,280.811871,114.302501,22.044867,NaN,Mv Arktis Star,NaN,NaN,mappingpiracy,1993-01-06 00:00:00+00:00


In [72]:
def norm_cat(s):
    if pd.isna(s): 
        return np.nan
    s = str(s).strip()
    return s.replace("_", " ").replace("-", " ").title()

for col in ["attack_type", "vessel_status", "vessel_type"]:
    if col in pir.columns:
        pir[col] = pir[col].map(lambda x: norm_cat(x))

# Map common null-like values to 'Unknown'
if "attack_type" in pir.columns:
    pir["attack_type"] = pir["attack_type"].replace({"Na": "Unknown", "N/A": "Unknown", "": "Unknown"}).fillna("Unknown")
if "vessel_status" in pir.columns:
    pir["vessel_status"] = pir["vessel_status"].replace({"Na": "Unknown", "N/A": "Unknown", "": "Unknown"}).fillna("Unknown")
if "vessel_type" in pir.columns:
    pir["vessel_type"] = pir["vessel_type"].fillna("Unknown")

# Country keys to ISO3 uppercase
for col in ["nearest_country", "eez_country"]:
    if col in pir.columns:
        pir[col] = pir[col].astype(str).str.upper().str.strip().replace({"NAN": np.nan})

pir[["attack_type","vessel_status","vessel_type"]].head()


,attack_type,vessel_status,vessel_type
0,Unknown,Unknown,Unknown
1,Unknown,Unknown,Unknown
2,Unknown,Unknown,Unknown
3,Unknown,Unknown,Unknown
4,Unknown,Unknown,Unknown


In [73]:
# join region info from country_codes.csv
codes["country"] = codes["country"].astype(str).str.upper().str.strip()
codes = codes.drop_duplicates(subset=["country"])

pir = pir.merge(
    codes[["country", "region"]],
    left_on="nearest_country", right_on="country", how="left"
).drop(columns=["country"]).rename(columns={"region": "nearest_region"})

pir["nearest_region"] = pir["nearest_region"].fillna("UNK")
pir.head(3)


,date,time,longitude,latitude,attack_type,location_description,nearest_country,eez_country,shore_distance,shore_longitude,shore_latitude,attack_description,vessel_name,vessel_type,vessel_status,data_source,timestamp,nearest_region
0,1993-01-02,NaN,116.9667,19.70,Unknown,Hong Kong - Luzon - Hainan,CHN,TWN,357.502373,115.825956,22.746644,NaN,Mv Cosmic Leader,Unknown,Unknown,mappingpiracy,1993-01-02 00:00:00+00:00,East Asia & Pacific
1,1993-01-04,NaN,116.0000,22.35,Unknown,Hong Kong - Luzon - Hainan,CHN,CHN,47.431573,115.825956,22.746644,NaN,Mv Tricolor Star III,Unknown,Unknown,mappingpiracy,1993-01-04 00:00:00+00:00,East Asia & Pacific
2,1993-01-06,NaN,115.2500,19.67,Unknown,Hong Kong - Luzon - Hainan,CHN,TWN,280.811871,114.302501,22.044867,NaN,Mv Arktis Star,Unknown,Unknown,mappingpiracy,1993-01-06 00:00:00+00:00,East Asia & Pacific


In [74]:
# denoising
# Year & month
pir["year"] = pir["date"].dt.year
pir["month"] = pir["date"].dt.month

# Shore distance sanity: >= 0; impute and cap extremes
if "shore_distance" in pir.columns:
    pir.loc[pir["shore_distance"] < 0, "shore_distance"] = np.nan
    if "nearest_region" in pir.columns:
        pir["shore_distance"] = pir.groupby("nearest_region")["shore_distance"] \
                                   .transform(lambda s: s.fillna(s.median()))
    pir["shore_distance"] = pir["shore_distance"].fillna(pir["shore_distance"].median())
    cap = pir["shore_distance"].quantile(0.99)
    pir.loc[pir["shore_distance"] > cap, "shore_distance"] = cap

# Remove the notorious (0,0) geocode if present
pir = pir[~((pir["longitude"] == 0) & (pir["latitude"] == 0))].reset_index(drop=True)

len(pir)


7490

In [75]:
# Minimum row guard (keeps project scale)
assert len(pir) >= MIN_ROWS, f"Only {len(pir)} rows remain (<{MIN_ROWS}). Loosen filters and re-run."

# Quick QA
print("Rows:", len(pir))
print("\nLongitude range:", pir["longitude"].min(), pir["longitude"].max())
print("Latitude range:", pir["latitude"].min(), pir["latitude"].max())
print("\nMissing (top 10):\n", pir.isna().sum().sort_values(ascending=False).head(10))

# Save
pir.to_csv(OUT_PATH, index=False)
print(f"\n✅ Saved cleaned file → {OUT_PATH}")


Rows: 7490

Longitude range: -163.9928 160.2646
Latitude range: -34.9033333 56.5878

Missing (top 10):
 time                    6342
attack_description      6318
vessel_name             1420
timestamp               1148
eez_country              294
nearest_country           18
location_description       8
date                       0
vessel_type                0
year                       0
dtype: int64



✅ Saved cleaned file → piracy/cleaned_data_output/cleaned_pirate_attacks.csv


In [76]:
# Attack types & trend by year
print(pir["attack_type"].value_counts())
trend = pir.groupby("year", as_index=False).size()
trend.tail()

attack_type
Boarded       3416
Attempted     1993
Boarding      1366
Hijacked       502
Unknown        120
Fired Upon      73
Suspicious      16
Explosion        3
Detained         1
Name: count, dtype: int64


,year,size
23,2016,186
24,2017,186
25,2018,201
26,2019,159
27,2020,190


In [77]:
# # Confirm pirate_attacks.csv has been cleaned

# ### 1. Count missing values per column
# # print(pirate_df.isnull().sum())

# ### 2. Check data types
# #       - date is datetime64
# #       - time is object or datetime.time
# #       - longitude/latitude are numeric (float/int) 
# # print(pirate_df.dtypes)

# ### 3. Check Value Ranges (Catch Outliers/Noise)
# #       - Longitude ∈ [−180, 180]
# #       - Latitude ∈ [−90, 90]
# #       - Shore distance ≥ 0
# print("Longitude range:", pirate_df['longitude'].min(), pirate_df['longitude'].max())
# print("Latitude range:", pirate_df['latitude'].min(), pirate_df['latitude'].max())
# print("Shore distance range:", pirate_df['shore_distance'].min(), pirate_df['shore_distance'].max())




In [78]:
# ### Basic Cleaning for country_indicators.csv ###
# # Drop duplicates
# indicators_df.drop_duplicates(inplace=True)

# # Handle missing numeric data (impute with median or drop)
# num_cols = indicators_df.select_dtypes(include='number').columns
# indicators_df[num_cols] = indicators_df[num_cols].fillna(indicators_df[num_cols].median())


In [79]:
# ### Cleaning for country_codes.csv ###
# # Drop duplicates
# codes_df.drop_duplicates(inplace=True)

# # Ensure consistency in ISO3 country codes
# codes_df['country'] = codes_df['country'].str.upper().str.strip()


<h3> EDA </h3>